
# 情景分析：在不同参数的情境下，交易组合估值的盈亏、希腊字母greeks的变化
- 思路：每一种情景下利用FDE估对一支产品估值$\rightarrow$（三层嵌套for)用于三维变量（指数点位、波动、到期时间）$\rightarrow$指数上涨1%，变化后FDE再估值$\rightarrow$新估值下盈亏（相减）、greeks变化、PNL分析


In [7]:
import pandas as pd
import numpy as np
import QuantLib as ql

In [8]:
#看涨期权定价（基于Quantlib包）   安装Quantlib:  pip install QuantLib-Python          文档：https://quantlib-python-docs.readthedocs.io/en/latest/pricing_engines.html#vanilla-options
def Pricing(price,vol,maturity):

    today = ql.Date(18, 11, 2020)
    ql.Settings.instance().evaluationDate = today
    #构建期权
    #普通看涨期权
    payoff=ql.PlainVanillaPayoff(ql.Option.Call, 102.0)
    #到期日期
    europeanExercise=ql.EuropeanExercise(ql.Date(maturity[0], maturity[1], maturity[2]))
    option = ql.EuropeanOption(payoff, europeanExercise)

    #输入参数——标的资产价格，无风险利率，标的资产波动率
    u = ql.SimpleQuote(price)      #标的资产价值
    r = ql.SimpleQuote(0.05)       #无风险利率
    sigma = ql.SimpleQuote(vol)    #波动率

    #假定无风险利率和波动率曲线是平的
    riskFreeCurve = ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(r), ql.Actual360())
    volatility = ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360())

    #初始化BS过程，并构造engine
    process = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                  ql.YieldTermStructureHandle(riskFreeCurve),
                                  ql.BlackVolTermStructureHandle(volatility))
    engine = ql.AnalyticEuropeanEngine(process)
    #对期权设定该engine
    option.setPricingEngine(engine)
#     print(f'看涨期权的当前价值为：{option.NPV():.4f}')
    return {'Value':option.NPV(),'Delta':option.delta(),'Gamma':option.gamma(),'Vega':option.vega()}


# 单笔合约损益、PNL
- 采用字典数据结构
- gamma cash有些问题

In [9]:
price_0 = 100  #标的价格期初
vol_0 = 0.10   #波动率期初
contract_mul = 100  #合约乘数
num_contracts = 10  #合约数量
value_0 = contract_mul*num_contracts*Pricing(price=100,vol=0.1,maturity=(18,5,2021))['Value'] #产品期初价值

price_l = [101,102,103]    #股价 上升1%，2%，3%
vol_l = [0.11,0.12,0.13]   #波动率 上升1%，2%，3%
maturity_l = [(18,11,2021),(18,5,2022),(18,11,2022)]



def Single_Product(price_l,vol_l,maturity_l):  #参数：标的价值变化列表，波动率变化列表、到期日变化列表
    key_l = []  #三维数据直接构造字典键值对
    pl_l = []
    delta_l = []
    gamma_l = []
    vega_l = []
    delta_cash_l = []
    gamma_cash_l = []
    for price in price_l:
        for vol in vol_l:
            for maturity in maturity_l:
                key_l.append((price,vol,maturity))#时间作为键，期权价值及greeks作为值
                
                #期权价值变化
                value = Pricing(price,vol,maturity)['Value']*contract_mul*num_contracts    #一种情景下，变化后的产品估值
                delta_v = value - value_0   #盈亏/价值变化
                pl_l.append(delta_v)
                
                #期权delta
                delta = Pricing(price,vol,maturity)['Delta']    
                delta_l.append(delta)
                
                #期权gamama
                gamma = Pricing(price,vol,maturity)['Gamma']    
                gamma_l.append(gamma)
                
                #期权vega
                vega = Pricing(price,vol,maturity)['Vega']    
                vega_l.append(vega)
                
                #一种情景下的delta cash
                delta_cash = 0
                #离散为小区间
                price_sec = []
                disp_nums = 100  #离散区间数
                for i in range(disp_nums):  
                    price_sec.append(price_0 + (i+1)*(price - price_0)/disp_nums)   #价格离散        
                    delta_p = Pricing(price_sec[i],vol,maturity)['Delta'] #小区间中，离散后的delta,Delta cash=合约数量*合约乘数*delta*变化的标的价格
                    delta_cash_i = delta_p*(price - price_0)/disp_nums*num_contracts*contract_mul   #离散后每一个小区间的delta cash
                    delta_cash = delta_cash + delta_cash_i
                delta_cash_l.append(delta_cash)   
                
                #一种情景下的gamma cash
                gamma_cash = 0
                #离散为小区间
                price_sec = []
                disp_nums = 100  #离散区间数
                for i in range(disp_nums):  
                    price_sec.append(price_0 + (i+1)*(price - price_0)/disp_nums)   #价格离散        
                    gamma_p = Pricing(price_sec[i],vol,maturity)['Gamma'] #小区间中，离散后的gamma
                    gamma_cash_i = 0.5*gamma_p*((price - price_0)/disp_nums)**2*num_contracts*contract_mul   #离散后每一个小区间的gamma cash.gamma cash =合约数量*合约乘数*0.5*gamma*变化的标的价格^2
                    gamma_cash = gamma_cash + gamma_cash_i
                gamma_cash_l.append(gamma_cash)
                
                               
                pl_dic = dict(zip(key_l,pl_l))   #键值合并，构造盈亏字典
                delta_dic = dict(zip(key_l,delta_l))   #键值合并，构造delta字典
                gamma_dic = dict(zip(key_l,gamma_l))
                vega_dic = dict(zip(key_l,vega_l))
                delta_cash_dic = dict(zip(key_l,delta_cash_l))
                gamma_cash_dic = dict(zip(key_l,gamma_cash_l))
                
                
                
                
                

    return {'PL':pl_dic,
            'Delta':delta_dic,
            'Gamma':gamma_dic,
            'Vega':vega_dic,
            'Delta_cash':delta_cash_dic,
            'Gamma_cash':gamma_cash_dic}
    

In [10]:
Single_Product(price_l=price_l,vol_l=vol_l,maturity_l=maturity_l)['PL']

{(101, 0.11, (18, 11, 2021)): 3591.1394011304615,
 (101, 0.11, (18, 5, 2022)): 6022.887972762988,
 (101, 0.11, (18, 11, 2022)): 8350.171145318722,
 (101, 0.12, (18, 11, 2021)): 3964.0172279715266,
 (101, 0.12, (18, 5, 2022)): 6452.469161317627,
 (101, 0.12, (18, 11, 2022)): 8817.309762729448,
 (101, 0.13, (18, 11, 2021)): 4340.372043566575,
 (101, 0.13, (18, 5, 2022)): 6889.095370014229,
 (101, 0.13, (18, 11, 2022)): 9295.53673913752,
 (102, 0.11, (18, 11, 2021)): 4271.4312157956065,
 (102, 0.11, (18, 5, 2022)): 6745.65664990187,
 (102, 0.11, (18, 11, 2022)): 9106.355576164968,
 (102, 0.12, (18, 11, 2021)): 4633.737774576493,
 (102, 0.12, (18, 5, 2022)): 7162.559308354703,
 (102, 0.12, (18, 11, 2022)): 9559.432274099376,
 (102, 0.13, (18, 11, 2021)): 5001.322235853328,
 (102, 0.13, (18, 5, 2022)): 7588.578160498093,
 (102, 0.13, (18, 11, 2022)): 10025.801296474887,
 (103, 0.11, (18, 11, 2021)): 4982.669132606936,
 (103, 0.11, (18, 5, 2022)): 7492.133809799036,
 (103, 0.11, (18, 11, 202

In [11]:
Single_Product(price_l=price_l,vol_l=vol_l,maturity_l=maturity_l)['Delta_cash']

{(101, 0.11, (18, 11, 2021)): 647.8785001701583,
 (101, 0.11, (18, 5, 2022)): 697.903042104798,
 (101, 0.11, (18, 11, 2022)): 736.044394519921,
 (101, 0.12, (18, 11, 2021)): 639.6904442427357,
 (101, 0.12, (18, 5, 2022)): 686.8700096087767,
 (101, 0.12, (18, 11, 2022)): 723.1611644899549,
 (101, 0.13, (18, 11, 2021)): 633.0019850872447,
 (101, 0.13, (18, 5, 2022)): 677.7400856445585,
 (101, 0.13, (18, 11, 2022)): 712.397368659902,
 (102, 0.11, (18, 11, 2021)): 1328.6548228301,
 (102, 0.11, (18, 5, 2022)): 1421.0433305807892,
 (102, 0.11, (18, 11, 2022)): 1492.5297827589523,
 (102, 0.12, (18, 11, 2021)): 1309.8603027215509,
 (102, 0.12, (18, 5, 2022)): 1397.3075177267342,
 (102, 0.12, (18, 11, 2022)): 1465.567302933951,
 (102, 0.13, (18, 11, 2021)): 1294.3706510441305,
 (102, 0.13, (18, 5, 2022)): 1377.548385611636,
 (102, 0.13, (18, 11, 2022)): 1442.929395203646,
 (103, 0.11, (18, 11, 2021)): 2040.669367002905,
 (103, 0.11, (18, 5, 2022)): 2168.115945112232,
 (103, 0.11, (18, 11, 2022)

In [12]:
Single_Product(price_l=price_l,vol_l=vol_l,maturity_l=maturity_l)['Gamma_cash']


{(101, 0.11, (18, 11, 2021)): 0.16668037277475123,
 (101, 0.11, (18, 5, 2022)): 0.12806287474523573,
 (101, 0.11, (18, 11, 2022)): 0.10380560034524684,
 (101, 0.12, (18, 11, 2021)): 0.15404027708947665,
 (101, 0.12, (18, 5, 2022)): 0.11926125088540669,
 (101, 0.12, (18, 11, 2022)): 0.09745114218765345,
 (101, 0.13, (18, 11, 2021)): 0.14308277144416143,
 (101, 0.13, (18, 5, 2022)): 0.1114350525017612,
 (101, 0.13, (18, 11, 2022)): 0.09162328320337555,
 (102, 0.11, (18, 11, 2021)): 0.6510057809214577,
 (102, 0.11, (18, 5, 2022)): 0.49952692823793776,
 (102, 0.11, (18, 11, 2022)): 0.40464072593155687,
 (102, 0.12, (18, 11, 2021)): 0.6032015868631951,
 (102, 0.12, (18, 5, 2022)): 0.46650025830787417,
 (102, 0.12, (18, 11, 2022)): 0.3809761829635223,
 (102, 0.13, (18, 11, 2021)): 0.5614298058084957,
 (102, 0.13, (18, 5, 2022)): 0.4368415118213249,
 (102, 0.13, (18, 11, 2022)): 0.359005945150031,
 (103, 0.11, (18, 11, 2021)): 1.4271087958132722,
 (103, 0.11, (18, 5, 2022)): 1.094572880154944